本项目主要来计算基于各空气污染物的监测水平，计算IAQI， 再基于IAQI计算整体空气质量指数（AQI）  
计算公式如下：  
>   $IAQI_{p}=\dfrac{(IAQI_{HI} - IAQI_{LO})}{(HP_{HI} - BP_{LO})}·(C_{p}-BP_{LO})+IAQI_{LO}$  
>   $AQI = MAX\{IAQI_{1}, ···\}$

In [40]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("/home/yuxk/mycode/MathModelng/Weather Forecast/")

data_path = "./附件1 监测点A空气质量预报基础数据.xlsx"
df = pd.read_excel(data_path, sheet_name=2)
df.iloc[131:135,:]

,监测日期,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3最大八小时滑动平均监测浓度(μg/m³),CO监测浓度(mg/m³)
131,2019-08-25,监测点A,9.0,27.0,50.0,39.0,70.0,0.8
132,2019-08-26,监测点A,5.0,24.0,20.0,14.0,60.0,0.6
133,2019-08-27,监测点A,4.0,26.0,NaN,9.0,56.0,0.6
134,2019-08-28,监测点A,6.0,32.0,42.0,20.0,170.0,0.6


In [49]:
table_pollu = [[0, 50, 150, 475, 800, 1600, 2100,2620],\
            [0, 40, 80, 180, 280, 565, 750, 940], \
            [0, 50, 150, 250, 350, 420, 500, 600], \
            [0, 35, 75, 115, 150, 250, 350, 500],\
            [0, 100, 160, 215, 265, 800], \
            [0, 2, 4, 14, 24, 36, 48, 60]]
aqi_sc = [0, 50, 100, 150, 200, 300, 400, 500]

def calcu_AQI(data, aqi_list, pollu_list):
    name = ['SO2', 'NO2', 'PM10', 'PM2.5', 'O3', 'CO']
    iaqi = []
    for p in range(len(data)):
        pos = 0
        if data[p] == 0:
            iaqi.append(0)
            continue
        for idx in range(len(pollu_list[p])):
            if data[p] < pollu_list[p][idx]:
                pos = idx
                break
        iaqi_h, iaqi_l,hp, bp = aqi_list[pos], aqi_list[pos-1], pollu_list[p][pos], pollu_list[p][pos-1] 
        iaqi_p = (iaqi_h - iaqi_l) / (hp - bp)*(data[p] - bp) + iaqi_l
        # print(iaqi_p)
        iaqi.append(int(iaqi_p) + 1)
    print(iaqi)
    aqi = max(iaqi)
    first_p = name[np.argmax(np.array(iaqi))]
    return (aqi, first_p)
        
        
# print(df["监测日期"][131])
data = df.iloc[131:135,:]
# print(data.head)
data.fillna(0, inplace=True)
# print(data)
res = []
for idx, row in data.iterrows():
    temp = calcu_AQI(row.iloc[2:8], aqi_sc, table_pollu)
    res.append(temp)
res
        

[10, 34, 51, 56, 36, 21]
[6, 31, 21, 21, 31, 16]
[5, 33, 0, 13, 29, 16]
[7, 41, 43, 29, 110, 16]


/tmp/ipykernel_872458/2247376743.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(0, inplace=True)


[(56, 'PM2.5'), (31, 'NO2'), (33, 'NO2'), (110, 'O3')]